#### Імпорт бібліотек

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType, DoubleType, TimestampType 
from datetime import date
import boto3 


VBox()

Starting Spark application


ID,Kind,State,Spark UI,Driver log,User,Current session?
24,pyspark,idle,Link,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Змінні:

In [2]:
# Paths to data
path_yellow = "s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/"
path_green = "s3://robot-dreams-source-data/home-work-1/nyc_taxi/green/"
path_zone_lookup = "s3://robot-dreams-source-data/home-work-1/nyc_taxi/taxi_zone_lookup.csv" 
base_out = "s3://rsherstiak-emr-studio/homework-1"
zones_csv = "s3://robot-dreams-source-data/home-work-1/nyc_taxi/taxi_zone_lookup.csv"
date_str = date.today().isoformat()  # YYYY-MM-DD


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Завдання 2: Імпорт, уніфікація та об’єднання

#### 1. Імпортувати дані жовтих та зелених поїздок:

In [3]:
 
# ----- read patches from bucket -----
 
bucket_green  = path_green.split("/")[2]
prefix_green  = "/".join(path_green.split("/")[3:])
bucket_yellow = path_yellow.split("/")[2]
prefix_yellow = "/".join(path_yellow.split("/")[3:])

s3 = boto3.client("s3")

# GREEN per-file -> cast -> union
resp_g = s3.list_objects_v2(Bucket=bucket_green, Prefix=prefix_green)
keys_g = [o["Key"] for o in resp_g.get("Contents", []) if o["Key"].endswith(".parquet")]

green_all = None
for k in sorted(keys_g):
    df = spark.read.parquet(f"s3://{bucket_green}/{k}")
    df_cast = df.select(
        F.col("VendorID").cast("long").alias("VendorID"),
        F.col("lpep_pickup_datetime").cast("timestamp_ntz").alias("lpep_pickup_datetime"),
        F.col("lpep_dropoff_datetime").cast("timestamp_ntz").alias("lpep_dropoff_datetime"),
        F.col("store_and_fwd_flag").cast("string").alias("store_and_fwd_flag"),
        F.col("RatecodeID").cast("double").alias("RatecodeID"),
        F.col("PULocationID").cast("long").alias("PULocationID"),
        F.col("DOLocationID").cast("long").alias("DOLocationID"),
        F.col("passenger_count").cast("double").alias("passenger_count"),
        F.col("trip_distance").cast("double").alias("trip_distance"),
        F.col("fare_amount").cast("double").alias("fare_amount"),
        F.col("extra").cast("double").alias("extra"),
        F.col("mta_tax").cast("double").alias("mta_tax"),
        F.col("tip_amount").cast("double").alias("tip_amount"),
        F.col("tolls_amount").cast("double").alias("tolls_amount"),
        F.col("ehail_fee").cast("double").alias("ehail_fee"),
        F.col("improvement_surcharge").cast("double").alias("improvement_surcharge"),
        F.col("total_amount").cast("double").alias("total_amount"),
        F.col("payment_type").cast("double").alias("payment_type"),
        F.col("trip_type").cast("double").alias("trip_type"),
        F.col("congestion_surcharge").cast("double").alias("congestion_surcharge"),
        F.lit(None).cast("double").alias("airport_fee")
    )
    green_all = df_cast if green_all is None else green_all.unionByName(df_cast, allowMissingColumns=True)

# YELLOW per-file -> cast -> union
resp_y = s3.list_objects_v2(Bucket=bucket_yellow, Prefix=prefix_yellow)
keys_y = [o["Key"] for o in resp_y.get("Contents", []) if o["Key"].endswith(".parquet")]

yellow_all = None
for k in sorted(keys_y):
    df = spark.read.parquet(f"s3://{bucket_yellow}/{k}")
    df_cast = df.select(
        F.col("VendorID").cast("long").alias("VendorID"),
        F.col("tpep_pickup_datetime").cast("timestamp_ntz").alias("tpep_pickup_datetime"),
        F.col("tpep_dropoff_datetime").cast("timestamp_ntz").alias("tpep_dropoff_datetime"),
        F.col("passenger_count").cast("double").alias("passenger_count"),
        F.col("trip_distance").cast("double").alias("trip_distance"),
        F.col("RatecodeID").cast("double").alias("RatecodeID"),
        F.col("store_and_fwd_flag").cast("string").alias("store_and_fwd_flag"),
        F.col("PULocationID").cast("long").alias("PULocationID"),
        F.col("DOLocationID").cast("long").alias("DOLocationID"),
        F.col("payment_type").cast("double").alias("payment_type"),
        F.col("fare_amount").cast("double").alias("fare_amount"),
        F.col("extra").cast("double").alias("extra"),
        F.col("mta_tax").cast("double").alias("mta_tax"),
        F.col("tip_amount").cast("double").alias("tip_amount"),
        F.col("tolls_amount").cast("double").alias("tolls_amount"),
        F.col("improvement_surcharge").cast("double").alias("improvement_surcharge"),
        F.col("total_amount").cast("double").alias("total_amount"),
        F.col("congestion_surcharge").cast("double").alias("congestion_surcharge"),
        F.col("airport_fee").cast("double").alias("airport_fee"),
        F.lit(None).cast("double").alias("ehail_fee"),
        F.lit(None).cast("double").alias("trip_type")
    )
    yellow_all = df_cast if yellow_all is None else yellow_all.unionByName(df_cast, allowMissingColumns=True) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)

#### 2. Уніфікувати схеми:

In [4]:
# ===== GREEN -> aligned schema via plain select =====
green_ready = green_all.select(
        # ids
        F.col("VendorID").cast("long").alias("VendorID"),
        # datetimes -> unified names
        F.col("lpep_pickup_datetime").cast("timestamp_ntz").alias("pickup_datetime"),
        F.col("lpep_dropoff_datetime").cast("timestamp_ntz").alias("dropoff_datetime"),
        # measures
        F.col("passenger_count").cast("double").alias("passenger_count"),
        F.col("trip_distance").cast("double").alias("trip_distance"),
        F.col("RatecodeID").cast("double").alias("RatecodeID"),
        F.col("store_and_fwd_flag").cast("string").alias("store_and_fwd_flag"),
        F.col("PULocationID").cast("long").alias("PULocationID"),
        F.col("DOLocationID").cast("long").alias("DOLocationID"),
        F.col("payment_type").cast("double").alias("payment_type"),
        F.col("fare_amount").cast("double").alias("fare_amount"),
        F.col("extra").cast("double").alias("extra"),
        F.col("mta_tax").cast("double").alias("mta_tax"),
        F.col("tip_amount").cast("double").alias("tip_amount"),
        F.col("tolls_amount").cast("double").alias("tolls_amount"),
        F.col("improvement_surcharge").cast("double").alias("improvement_surcharge"),
        F.col("total_amount").cast("double").alias("total_amount"),
        F.col("congestion_surcharge").cast("double").alias("congestion_surcharge"),
        # fields missing in green -> nulls with correct types
        F.lit(None).cast("double").alias("airport_fee"),
        # green-only fields kept
        F.col("ehail_fee").cast("double").alias("ehail_fee"),
        F.col("trip_type").cast("double").alias("trip_type"),
        # optional marker
        F.lit("green").alias("service_type")
).withColumn("taxi_type", F.lit("green"))

# ===== YELLOW -> aligned schema via plain select =====
yellow_ready = yellow_all.select(
        # ids
        F.col("VendorID").cast("long").alias("VendorID"),
        # datetimes -> unified names
        F.col("tpep_pickup_datetime").cast("timestamp_ntz").alias("pickup_datetime"),
        F.col("tpep_dropoff_datetime").cast("timestamp_ntz").alias("dropoff_datetime"),
        # measures
        F.col("passenger_count").cast("double").alias("passenger_count"),
        F.col("trip_distance").cast("double").alias("trip_distance"),
        F.col("RatecodeID").cast("double").alias("RatecodeID"),
        F.col("store_and_fwd_flag").cast("string").alias("store_and_fwd_flag"),
        F.col("PULocationID").cast("long").alias("PULocationID"),
        F.col("DOLocationID").cast("long").alias("DOLocationID"),
        F.col("payment_type").cast("double").alias("payment_type"),
        F.col("fare_amount").cast("double").alias("fare_amount"),
        F.col("extra").cast("double").alias("extra"),
        F.col("mta_tax").cast("double").alias("mta_tax"),
        F.col("tip_amount").cast("double").alias("tip_amount"),
        F.col("tolls_amount").cast("double").alias("tolls_amount"),
        F.col("improvement_surcharge").cast("double").alias("improvement_surcharge"),
        F.col("total_amount").cast("double").alias("total_amount"),
        F.col("congestion_surcharge").cast("double").alias("congestion_surcharge"),
        # yellow field present
        F.col("airport_fee").cast("double").alias("airport_fee"),
        # fields missing in yellow -> nulls with correct types
        F.lit(None).cast("double").alias("ehail_fee"),
        F.lit(None).cast("double").alias("trip_type"),
        # optional marker
        F.lit("yellow").alias("service_type")
).withColumn("taxi_type", F.lit("yellow"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 3. Об’єднати дані в один датафрейм raw_trips_df.

In [5]:
# Union into one final DataFrame
raw_trips_df  = green_ready.unionByName(yellow_ready, allowMissingColumns=True)

# Optionally check schema and row count
raw_trips_df.show(3, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+---------+------------+---------+
|VendorID|pickup_datetime    |dropoff_datetime   |passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|ehail_fee|trip_type|service_type|taxi_type|
+--------+-------------------+-------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+---------+---------+------------+---------+
|2       |2014-01-01 00:17:26|2014-01-01 00:37:11|1.0            |2

#### 4. Виконати фільтрацію аномальних записів: * Вилучити поїздки з відстанню < 0.1 км, тарифом < $2, тривалістю < 1 хв.
  

In [6]:
filtered_trips_df = raw_trips_df.filter(
    (F.col("trip_distance") >= 0.1) &
    (F.col("total_amount")  >= 2.0) &
    (
        (
            F.col("dropoff_datetime").cast("timestamp").cast("long")
            - F.col("pickup_datetime").cast("timestamp").cast("long")
        ) / 60.0 >= 1.0
    )
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 5. Додати наступні колонки:   * pickup_hour   * pickup_day_of_week   * duration_min 

In [7]:
featured_trips_df = filtered_trips_df.select(
    "*",
    F.hour("pickup_datetime").alias("pickup_hour"),
    F.date_format("pickup_datetime", "EEEE").alias("pickup_day_of_week"),
    (
        (
            F.col("dropoff_datetime").cast("timestamp").cast("long")
            - F.col("pickup_datetime").cast("timestamp").cast("long")
        ).cast("double") / F.lit(60.0)
    ).alias("duration_min")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 6. Виконати JOIN з taxi_zone_lookup.csv, додавши поля pickup_zone, dropoff_zone.

In [8]:
# Read zones
zones_df = (spark.read
    .option("header", "true")
    .csv(zones_csv)
    .select(
        F.col("LocationID").cast("int").alias("LocationID"),
        F.col("Zone").alias("Zone")
    )
)

# Join zones -> final RAF-TRIPS-DF
raf_trips_df = (featured_trips_df
    .join(
        zones_df.select(
            F.col("LocationID").alias("PULocationID"),
            F.col("Zone").alias("pickup_zone")
        ),
        on="PULocationID", how="left"
    )
    .join(
        zones_df.select(
            F.col("LocationID").alias("DOLocationID"),
            F.col("Zone").alias("dropoff_zone")
        ),
        on="DOLocationID", how="left"
    )
)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Завдання 3: Створення та агрегація фінального датафрейму zone_summary

####  1. Створити датафрейм zone_summary

In [9]:
#   * pickup_zone — зона посадки
#   * total_trips — загальна кількість поїздок
#   * avg_trip_distance — середня відстань поїздки
#   * avg_total_amount — середній тариф поїздки
#   * avg_tip_amount — середня сума чайових
#   * yellow_share — частка жовтих поїздок у зоні
#   * green_share — частка зелених поїздок у зоні
#   * max_trip_distance — максимальна відстань поїздки
#   * min_tip_amount — мінімальна сума чайових 

# Aggregates
total_cnt  = F.count(F.lit(1))
yellow_cnt = F.sum(F.when(F.col("taxi_type") == "yellow", 1).otherwise(0))
green_cnt  = F.sum(F.when(F.col("taxi_type") == "green",  1).otherwise(0))

zone_summary = (raf_trips_df.groupBy("pickup_zone")
    .agg(
        total_cnt.alias("total_trips"),
        F.avg("trip_distance").alias("avg_trip_distance"),
        F.avg("total_amount").alias("avg_total_amount"),
        F.avg("tip_amount").alias("avg_tip_amount"),
        (yellow_cnt/total_cnt).alias("yellow_share"),
        (green_cnt/total_cnt).alias("green_share"),
        F.max("trip_distance").alias("max_trip_distance"),
        F.min("tip_amount").alias("min_tip_amount")
    )
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
zone_summary.show(3, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-----------+------------------+------------------+------------------+-------------------+------------------+-----------------+--------------+
|pickup_zone   |total_trips|avg_trip_distance |avg_total_amount  |avg_tip_amount    |yellow_share       |green_share       |max_trip_distance|min_tip_amount|
+--------------+-----------+------------------+------------------+------------------+-------------------+------------------+-----------------+--------------+
|Westerleigh   |1254       |8.929792663476873 |39.18330143540672 |2.514744816586922 |0.48564593301435405|0.5143540669856459|124.4            |0.0           |
|Pelham Parkway|97611      |16.476772802245637|18.982729405497224|0.5824950056858349|0.10123858991302209|0.8987614100869779|221944.04        |0.0           |
|Rego Park     |702740     |4.957281483905853 |16.124224734610422|0.799293878248    |0.09993454193585109|0.900065458064149 |185845.97        |0.0           |
+--------------+-----------+------------------+-----

#### 2. Зберегти агрегаційні результати:

In [11]:
zone_summary_out = f"{base_out}/results/zone_statistic/{date_str}/zone_statistic.parquet"
(zone_summary.coalesce(1).write.mode("overwrite").parquet(zone_summary_out))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Завдання 4: Агрегація по днях тижня та зонах

#### 1. Підрахувати кількість поїздок за певний день тижня (наприклад, понеділок — Sunday, etc.).

In [12]:
total_cnt = F.count(F.lit(1))
high_cnt  = F.sum(F.when(F.col("total_amount") > 30.0, 1).otherwise(0))

zone_days_statstic = (
    raf_trips_df
    .groupBy("pickup_zone", "pickup_day_of_week")  # weekday already prepared earlier
    .agg(
        total_cnt.alias("total_trips"),
        (high_cnt / total_cnt).alias("high_fare_share")
    )
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 2. Розрахувати частку поїздок з тарифами більше $30 за кожною зоною (high_fare_share).

In [13]:
zone_days_statstic.show(10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+------------------+-----------+-------------------+
|pickup_zone              |pickup_day_of_week|total_trips|high_fare_share    |
+-------------------------+------------------+-----------+-------------------+
|Stapleton                |Wednesday         |630        |0.3047619047619048 |
|Astoria                  |Tuesday           |574342     |0.06358580775914002|
|Bedford                  |Monday            |135201     |0.09459989201263304|
|Murray Hill-Queens       |Monday            |3339       |0.40041928721174   |
|Auburndale               |Friday            |2586       |0.3008507347254447 |
|Glen Oaks                |Monday            |1714       |0.5694282380396732 |
|Brooklyn Heights         |Tuesday           |357545     |0.1303919786320603 |
|Forest Park/Highland Park|Wednesday         |776        |0.28994845360824745|
|Bronx Park               |Wednesday         |3867       |0.23972071373157486|
|Van Cortlandt Village    |Saturday          |18073 

#### 3. Зберегти файл в у S3: /your-bucket/results/zone_days_statstic/{date}/zone_days_statstic.parquet.

In [14]:
zone_days_out = f"{base_out}/results/zone_days_statstic/{date_str}/zone_days_statstic.parquet"

(zone_days_statstic
 .coalesce(1)
 .write.mode("overwrite")
 .parquet(zone_days_out))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
spark.catalog.clearCache()   # clear cached tables
spark.stop()                 # stop SparkSession

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…